In [42]:
from transformers import pipeline
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [43]:
!pip install -q kaggle

In [44]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"deltaburs","key":"83485bda236d1b3fe06ec4121012ec46"}'}

In [45]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [46]:
!cp kaggle.json ~/.kaggle

In [47]:
! chmod 600 ~/.kaggle/kaggle.json

In [48]:
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

twitter-entity-sentiment-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)


In [49]:
import zipfile
zip_ref = zipfile.ZipFile('/content/twitter-entity-sentiment-analysis.zip')
zip_ref.extractall()
zip_ref.close()

In [50]:
column=['Tweet ID', 'entity', 'sentiment', 'Tweet content']

In [51]:
df = pd.read_csv('/content/twitter_training.csv', names=column)
df

,Tweet ID,entity,sentiment,Tweet content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [52]:
df.sentiment.value_counts()

Negative      22542
Positive      20832
Neutral       18318
Irrelevant    12990
Name: sentiment, dtype: int64

In [53]:
df = df[df.sentiment != 'Neutral'].copy()

In [54]:
df.sentiment.value_counts()

Negative      22542
Positive      20832
Irrelevant    12990
Name: sentiment, dtype: int64

In [55]:
df = df[df.sentiment != 'Irrelevant'].copy()

In [56]:
df.sentiment.value_counts()

Negative    22542
Positive    20832
Name: sentiment, dtype: int64

In [57]:
df.dropna(inplace=True)

In [58]:
df.isnull().sum()

Tweet ID         0
entity           0
sentiment        0
Tweet content    0
dtype: int64

In [59]:
target_map = {'Positive':1, 'Negative':0}
df['target'] = df['sentiment'].map(target_map)

In [60]:
df.drop(['Tweet ID', 'entity'], axis=1, inplace=True)
df.head()

,sentiment,Tweet content,target
0,Positive,im getting on borderlands and i will murder yo...,1
1,Positive,I am coming to the borders and I will kill you...,1
2,Positive,im getting on borderlands and i will kill you ...,1
3,Positive,im coming on borderlands and i will murder you...,1
4,Positive,im getting on borderlands 2 and i will murder ...,1


In [61]:
df.isnull().sum()

sentiment        0
Tweet content    0
target           0
dtype: int64

In [105]:
df.sentiment.value_counts()

Negative    22358
Positive    20655
Name: sentiment, dtype: int64

In [62]:
text = df['Tweet content'].tolist()

In [101]:
df.iloc[120, -1]

1

In [102]:
text[120]

'Jumping on stream early today! Playing some more of the Borderlands DLCs & vibing out to my new playlists I made .  come vibe. Also dropping my quality down to 720p to see if it makes any difference to how well my stream runs on my dumb internet lmao . . . twitch.tv/heyitscece_'

In [63]:
# Use a pipeline as a high-level helper
from transformers import pipeline

clf = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [64]:
from tqdm import tqdm

In [94]:
for i in tqdm(range(int(1)), ncols=100):
  prediction = clf(text[:100])
prediction

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.90s/it]


[{'label': 'NEGATIVE', 'score': 0.9834704995155334},
 {'label': 'POSITIVE', 'score': 0.9932871460914612},
 {'label': 'POSITIVE', 'score': 0.978373646736145},
 {'label': 'NEGATIVE', 'score': 0.9768757820129395},
 {'label': 'POSITIVE', 'score': 0.7018905282020569},
 {'label': 'NEGATIVE', 'score': 0.9805620312690735},
 {'label': 'NEGATIVE', 'score': 0.7845362424850464},
 {'label': 'NEGATIVE', 'score': 0.9357827305793762},
 {'label': 'POSITIVE', 'score': 0.9987744688987732},
 {'label': 'NEGATIVE', 'score': 0.9372897148132324},
 {'label': 'NEGATIVE', 'score': 0.9648420810699463},
 {'label': 'POSITIVE', 'score': 0.7795966267585754},
 {'label': 'POSITIVE', 'score': 0.9994412064552307},
 {'label': 'POSITIVE', 'score': 0.999355137348175},
 {'label': 'POSITIVE', 'score': 0.9994412064552307},
 {'label': 'POSITIVE', 'score': 0.9996585845947266},
 {'label': 'POSITIVE', 'score': 0.9978618025779724},
 {'label': 'NEGATIVE', 'score': 0.9911395907402039},
 {'label': 'NEGATIVE', 'score': 0.99786877632141

In [106]:
clf('This is the best whay to learn python')

[{'label': 'POSITIVE', 'score': 0.9996070265769958}]